- So let's pull the tournament data over the last... 5? 10? 15? years? (data only goes back to 2003)
- Compare higher seed v. lower seed
- Whether the higher seed won or loss is our predictive value
- Build the model off of that

- Start with the compact NCAA tourney scores
- Pull the teamIDs
- Link them to the year calculated databases
- Pull the seed, pull the mean absolute rank (in case of seed ties)
- Label higher seed/lower seed IDs
- Check if higher seed is WTeamID
- Calculate the difference? Or aggregate the data?
- Maybe do both, if I don't hate myself later, see which model is better
- From that, build the models

In [2]:
import pandas as pd
import numpy as np

In [3]:
tourney = pd.read_csv('./DataFiles/NCAATourneyCompactResults.csv')

In [275]:
def gen_higher_seed(tourney_game, season_data, year):
    team1 = tourney_game['WTeamID']
    team2 = tourney_game['LTeamID']
    seed1 = season_data[season_data['TeamID'] == team1]['Seed'].values[0]
    seed2 = season_data[season_data['TeamID'] == team2]['Seed'].values[0]
    matchup = {}
    if seed1 < seed2:
        matchup['FavoriteID'] = team1
        matchup['UnderdogID'] = team2
        matchup['ExpectedWin'] = 1
    elif seed1 == seed2:
        rank1 = season_data[season_data['TeamID'] == team1]['AbsMeanRank'].values[0]
        rank2 = season_data[season_data['TeamID'] == team2]['AbsMeanRank'].values[0]
        if rank1 < rank2:
            matchup['FavoriteID'] = team1
            matchup['UnderdogID'] = team2
            matchup['ExpectedWin'] = 1
        else:
            matchup['FavoriteID'] = team2
            matchup['UnderdogID'] = team1
            matchup['ExpectedWin'] = 0
    else:
        matchup['FavoriteID'] = team2
        matchup['UnderdogID'] = team1
        matchup['ExpectedWin'] = 0
    df_matchup = pd.DataFrame(columns=['FavoriteID', 'UnderdogID', 'ExpectedWin'])
    df_matchup.loc[str(year) + "_" + str(matchup['FavoriteID']) + "_" + str(matchup['UnderdogID'])] = matchup
    return df_matchup

In [17]:
tourney[tourney['Season'] == 2017].head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
2050,2017,134,1243,95,1448,88,N,0
2051,2017,134,1291,67,1309,66,N,0
2052,2017,135,1413,67,1300,63,N,0
2053,2017,135,1425,75,1344,71,N,0
2054,2017,136,1112,100,1315,82,N,0


In [22]:
stat2017 = pd.read_csv('./GeneratedDatasets/generated2017.csv')

In [23]:
gen_higher_seed(tourney.loc[2050], stat2017, 2017)

,FavoriteID,UnderdogID,ExpectedWin
144812432017,1448,1243,0


In [32]:
def get_stat_model(year):
    return pd.read_csv('./GeneratedDatasets/generated' + str(year) + '.csv')

In [169]:
def gen_tourney_model_base(year):
    tournament = tourney[tourney['Season'] == year].copy()
    stat_model = get_stat_model(year)
    ncaamodel = pd.DataFrame(columns=['FavoriteID', 'UnderdogID', 'ExpectedWin'])
    for gm in tournament.index:
        ncaamodel = pd.concat([ncaamodel, gen_higher_seed(tourney.loc[gm], stat_model, year)])
    return ncaamodel

In [170]:
gen_tourney_model_base(2017)

,FavoriteID,UnderdogID,ExpectedWin
144812432017,1448,1243,0
130912912017,1309,1291,0
130014132017,1300,1413,0
142513442017,1425,1344,1
111213152017,1112,1315,1
113914572017,1139,1457,1
119611902017,1196,1190,1
119911952017,1199,1195,1
121113552017,1211,1355,1
123513052017,1235,1305,1


In [277]:
def gen_aggregate(team1_id, team2_id, year):
    stat_model = get_stat_model(year)
    fav_team = stat_model[stat_model['TeamID'] == team1_id].drop(['TeamID', 'TeamName'], axis=1).reset_index(drop = True)
    und_team = stat_model[stat_model['TeamID'] == team2_id].drop(['TeamID', 'TeamName'], axis=1).reset_index(drop = True)
    new_id = str(year) + "_" + str(team1_id) + "_" + str(team2_id)
    aggregate = fav_team.join(und_team, lsuffix="Fav", rsuffix="Und")
    aggregate.loc[0, "NewID"] = new_id
    aggregate.set_index("NewID", inplace = True)
    del aggregate.index.name
    return aggregate

In [84]:
gen_aggregate(1400, 1328, 2018)

,WinsFav,LossesFav,PercentageFav,MeanRankFav,AbsMeanRankFav,SeedFav,coach_expFav,made_tournamentFav,first_roundFav,second_roundFav,...,AgstOppOffAvgOppFGA3Und,AgstOppOffAvgOppFTMUnd,AgstOppOffAvgOppFTAUnd,AgstOppOffAvgOppORUnd,AgstOppOffAvgOppDRUnd,AgstOppOffAvgOppAstUnd,AgstOppOffAvgOppTOUnd,AgstOppOffAvgOppStlUnd,AgstOppOffAvgOppBlkUnd,AgstOppOffAvgOppPFUnd
140013282018,19,14,0.576,41.1311,41,10,9,6,6,3,...,3.574,-1.486,-1.943,1.217,1.878,0.383,0.711,0.605,0.359,-0.153


In [276]:
def gen_differential(team1_id, team2_id, year):
    stat_model = get_stat_model(year)
    fav_team = stat_model[stat_model['TeamID'] == team1_id].drop(['TeamID', 'TeamName'], axis=1).reset_index(drop = True)
    und_team = stat_model[stat_model['TeamID'] == team2_id].drop(['TeamID', 'TeamName'], axis=1).reset_index(drop = True)
    differential = {}
    diff_col = []
    int_col = []
    for col in fav_team.columns:
        if type(fav_team.loc[0, col]) is np.int64:
            int_col.append(col + "Diff")
        differential[col + "Diff"] = round((fav_team.loc[0, col] - und_team.loc[0, col]), 3)
        diff_col.append(col + "Diff")
    df_diff = pd.DataFrame(columns=diff_col)
    df_diff.loc[str(year) + "_" + str(team1_id) + "_" + str(team2_id)] = differential
    for col in df_diff.columns:
        if col in int_col:
            df_diff[col] = df_diff[col].astype(int)
    return df_diff

In [274]:
gen_differential(1400, 1328, 2018)

,WinsDiff,LossesDiff,PercentageDiff,MeanRankDiff,AbsMeanRankDiff,SeedDiff,coach_expDiff,made_tournamentDiff,first_roundDiff,second_roundDiff,...,AgstOppOffAvgOppFGA3Diff,AgstOppOffAvgOppFTMDiff,AgstOppOffAvgOppFTADiff,AgstOppOffAvgOppORDiff,AgstOppOffAvgOppDRDiff,AgstOppOffAvgOppAstDiff,AgstOppOffAvgOppTODiff,AgstOppOffAvgOppStlDiff,AgstOppOffAvgOppBlkDiff,AgstOppOffAvgOppPFDiff
140013282018,1,1,-0.005,0.754,2,0,-21,-11,-11,-7,...,-4.886,-1.66,-2.368,-1.531,-1.748,-2.763,-0.71,-1.651,-1.474,0.065


In [166]:
gen_aggregate(1400, 1328, 2018).join(gen_differential(1400, 1328, 2018))

,WinsFav,LossesFav,PercentageFav,MeanRankFav,AbsMeanRankFav,SeedFav,coach_expFav,made_tournamentFav,first_roundFav,second_roundFav,...,AgstOppOffAvgOppFGA3Diff,AgstOppOffAvgOppFTMDiff,AgstOppOffAvgOppFTADiff,AgstOppOffAvgOppORDiff,AgstOppOffAvgOppDRDiff,AgstOppOffAvgOppAstDiff,AgstOppOffAvgOppTODiff,AgstOppOffAvgOppStlDiff,AgstOppOffAvgOppBlkDiff,AgstOppOffAvgOppPFDiff
140013282018,19,14,0.576,41.1311,41,10,9,6,6,3,...,-4.886,-1.66,-2.368,-1.531,-1.748,-2.763,-0.71,-1.651,-1.474,0.065


In [258]:
def gen_tourney_model_agg(year):
    tournament = tourney[tourney['Season'] == year].copy().reset_index(drop = True)
    stat_model = get_stat_model(year)
    df = gen_higher_seed(tournament.loc[0], stat_model, year)
    df_agg = gen_aggregate(df['FavoriteID'].values[0], df['UnderdogID'].values[0], year)
    ncaamodel = df.join(df_agg)
    for gm in range(1, len(tournament)):
        df = gen_higher_seed(tournament.loc[gm], stat_model, year)
        df_agg = gen_aggregate(df['FavoriteID'].values[0], df['UnderdogID'].values[0], year)
        ncaamodel = pd.concat([ncaamodel, df.join(df_agg)])
    return ncaamodel

In [278]:
gen_tourney_model_agg(2017)

,FavoriteID,UnderdogID,ExpectedWin,WinsFav,LossesFav,PercentageFav,MeanRankFav,AbsMeanRankFav,SeedFav,coach_expFav,...,AgstOppOffAvgOppFGA3Und,AgstOppOffAvgOppFTMUnd,AgstOppOffAvgOppFTAUnd,AgstOppOffAvgOppORUnd,AgstOppOffAvgOppDRUnd,AgstOppOffAvgOppAstUnd,AgstOppOffAvgOppTOUnd,AgstOppOffAvgOppStlUnd,AgstOppOffAvgOppBlkUnd,AgstOppOffAvgOppPFUnd
2017_1448_1243,1448,1243,0,19,13,0.594,39.1857,35,11,5,...,-1.691,-1.925,-3.222,0.115,-2.295,-0.941,2.362,-0.390,-0.716,0.709
2017_1309_1291,1309,1291,0,17,11,0.607,177.5143,182,16,5,...,-4.330,-2.015,-1.826,0.943,1.179,-1.620,1.370,-0.317,-0.381,-0.429
2017_1300_1413,1300,1413,0,22,8,0.733,165.4714,164,16,8,...,0.142,-0.696,-1.047,-0.014,-0.703,0.493,0.883,1.090,0.413,1.818
2017_1425_1344,1425,1344,1,24,9,0.727,50.5286,48,11,6,...,-1.667,-3.150,-3.628,-0.616,0.243,-2.053,1.323,-0.032,-0.601,0.694
2017_1112_1315,1112,1315,1,30,4,0.882,12.0286,10,2,13,...,-3.259,0.585,1.165,0.526,0.413,-1.282,1.982,0.793,-0.608,0.782
2017_1139_1457,1139,1457,1,23,8,0.742,21.8857,23,4,6,...,1.279,1.072,0.694,1.852,1.298,-2.157,0.494,-0.087,0.281,1.013
2017_1196_1190,1196,1190,1,24,8,0.750,11.6857,9,4,6,...,-1.299,1.222,1.541,-0.879,-1.807,-3.065,2.897,0.629,0.612,1.620
2017_1199_1195,1199,1195,1,25,8,0.758,16.9714,18,3,29,...,-0.374,-0.431,-0.344,0.383,-3.654,-1.431,-1.585,0.131,-0.150,-1.390
2017_1211_1355,1211,1355,1,32,1,0.970,1.9571,2,1,18,...,6.498,-3.044,-4.468,0.662,-2.011,1.366,-1.578,0.782,-0.187,1.291
2017_1235_1305,1235,1305,1,23,10,0.697,20.0000,21,5,6,...,1.622,-3.109,-3.905,-0.195,-0.803,0.427,-1.591,-0.507,-1.326,1.509


In [260]:
def gen_tourney_model_diff(year):
    tournament = tourney[tourney['Season'] == year].copy().reset_index(drop = True)
    stat_model = get_stat_model(year)
    df = gen_higher_seed(tournament.loc[0], stat_model, year)
    df_diff = gen_differential(df['FavoriteID'].values[0], df['UnderdogID'].values[0], year)
    ncaamodel = df.join(df_diff)
    for gm in range(1, len(tournament)):
        df = gen_higher_seed(tournament.loc[gm], stat_model, year)
        df_diff = gen_differential(df['FavoriteID'].values[0], df['UnderdogID'].values[0], year)
        ncaamodel = pd.concat([ncaamodel, df.join(df_diff)])
    return ncaamodel

In [279]:
gen_tourney_model_diff(2017)

,FavoriteID,UnderdogID,ExpectedWin,WinsDiff,LossesDiff,PercentageDiff,MeanRankDiff,AbsMeanRankDiff,SeedDiff,coach_expDiff,...,AgstOppOffAvgOppFGA3Diff,AgstOppOffAvgOppFTMDiff,AgstOppOffAvgOppFTADiff,AgstOppOffAvgOppORDiff,AgstOppOffAvgOppDRDiff,AgstOppOffAvgOppAstDiff,AgstOppOffAvgOppTODiff,AgstOppOffAvgOppStlDiff,AgstOppOffAvgOppBlkDiff,AgstOppOffAvgOppPFDiff
2017_1448_1243,1448,1243,0,-1,0,-0.012,-1.086,-2,0,-14,...,4.143,3.719,5.440,-0.303,0.801,1.035,-3.208,0.444,0.826,2.952
2017_1309_1291,1309,1291,0,-2,-4,0.048,-21.414,-20,0,0,...,4.871,0.415,0.050,-0.099,-4.270,0.791,0.241,2.559,0.662,1.749
2017_1300_1413,1300,1413,0,2,-4,0.108,-25.786,-31,0,-7,...,-0.228,-2.813,-3.045,1.045,-0.328,-2.246,-1.138,-1.938,-0.903,-3.494
2017_1425_1344,1425,1344,1,4,-3,0.102,-2.843,-4,0,-5,...,3.779,-0.336,-1.419,1.889,-1.690,2.114,-1.216,-0.179,-0.029,0.153
2017_1112_1315,1112,1315,1,11,-5,0.203,-137.971,-140,-13,4,...,3.387,-3.245,-4.701,-1.448,-6.000,-0.737,-2.663,-2.130,-0.466,0.901
2017_1139_1457,1139,1457,1,-1,2,-0.058,-71.200,-67,-9,1,...,-2.063,-1.526,-1.059,-3.119,-4.404,-2.022,0.230,-1.332,-1.218,-0.392
2017_1196_1190,1196,1190,1,-1,1,-0.031,-53.257,-54,-9,4,...,-1.216,-3.207,-3.912,0.585,0.672,-0.721,-0.659,-1.341,-1.717,0.422
2017_1199_1195,1199,1195,1,2,1,-0.009,-77.272,-73,-11,21,...,0.255,1.273,1.932,0.726,1.241,-0.920,3.910,-1.264,-0.728,3.369
2017_1211_1355,1211,1355,1,16,-15,0.470,-173.486,-179,-15,17,...,-8.283,-0.099,0.959,-0.008,-1.569,-5.439,1.844,-1.196,-0.338,-0.398
2017_1235_1305,1235,1305,1,-5,4,-0.127,-24.657,-23,-7,4,...,-1.033,-0.803,-1.504,0.946,2.674,-1.526,3.080,-2.024,0.867,-3.742


In [271]:
def gen_tourney_model_aggdiff(year):
    df_agg = gen_tourney_model_agg(year)
    df_diff = gen_tourney_model_diff(year)
    df = df_agg.join(df_diff, rsuffix="_drop")
    return df.drop(["FavoriteID_drop", "UnderdogID_drop", "ExpectedWin_drop"], axis=1)

In [280]:
gen_tourney_model_aggdiff(2017)

,FavoriteID,UnderdogID,ExpectedWin,WinsFav,LossesFav,PercentageFav,MeanRankFav,AbsMeanRankFav,SeedFav,coach_expFav,...,AgstOppOffAvgOppFGA3Diff,AgstOppOffAvgOppFTMDiff,AgstOppOffAvgOppFTADiff,AgstOppOffAvgOppORDiff,AgstOppOffAvgOppDRDiff,AgstOppOffAvgOppAstDiff,AgstOppOffAvgOppTODiff,AgstOppOffAvgOppStlDiff,AgstOppOffAvgOppBlkDiff,AgstOppOffAvgOppPFDiff
2017_1448_1243,1448,1243,0,19,13,0.594,39.1857,35,11,5,...,4.143,3.719,5.440,-0.303,0.801,1.035,-3.208,0.444,0.826,2.952
2017_1309_1291,1309,1291,0,17,11,0.607,177.5143,182,16,5,...,4.871,0.415,0.050,-0.099,-4.270,0.791,0.241,2.559,0.662,1.749
2017_1300_1413,1300,1413,0,22,8,0.733,165.4714,164,16,8,...,-0.228,-2.813,-3.045,1.045,-0.328,-2.246,-1.138,-1.938,-0.903,-3.494
2017_1425_1344,1425,1344,1,24,9,0.727,50.5286,48,11,6,...,3.779,-0.336,-1.419,1.889,-1.690,2.114,-1.216,-0.179,-0.029,0.153
2017_1112_1315,1112,1315,1,30,4,0.882,12.0286,10,2,13,...,3.387,-3.245,-4.701,-1.448,-6.000,-0.737,-2.663,-2.130,-0.466,0.901
2017_1139_1457,1139,1457,1,23,8,0.742,21.8857,23,4,6,...,-2.063,-1.526,-1.059,-3.119,-4.404,-2.022,0.230,-1.332,-1.218,-0.392
2017_1196_1190,1196,1190,1,24,8,0.750,11.6857,9,4,6,...,-1.216,-3.207,-3.912,0.585,0.672,-0.721,-0.659,-1.341,-1.717,0.422
2017_1199_1195,1199,1195,1,25,8,0.758,16.9714,18,3,29,...,0.255,1.273,1.932,0.726,1.241,-0.920,3.910,-1.264,-0.728,3.369
2017_1211_1355,1211,1355,1,32,1,0.970,1.9571,2,1,18,...,-8.283,-0.099,0.959,-0.008,-1.569,-5.439,1.844,-1.196,-0.338,-0.398
2017_1235_1305,1235,1305,1,23,10,0.697,20.0000,21,5,6,...,-1.033,-0.803,-1.504,0.946,2.674,-1.526,3.080,-2.024,0.867,-3.742


In [281]:
ncaa_agg = gen_tourney_model_agg(2003)
ncaa_diff = gen_tourney_model_diff(2003)
ncaa_aggdiff = gen_tourney_model_aggdiff(2003)
for yr in range(2004, 2018):
    ncaa_agg = pd.concat([ncaa_agg, gen_tourney_model_agg(yr)])
    ncaa_diff = pd.concat([ncaa_diff, gen_tourney_model_diff(yr)])
    ncaa_aggdiff = pd.concat([ncaa_aggdiff, gen_tourney_model_aggdiff(yr)])

In [358]:
ncaa_agg.to_csv('./GeneratedDatasets/NCAA-Aggregate.csv', index_label='ID')
ncaa_diff.to_csv('./GeneratedDatasets/NCAA-Differential.csv', index_label='ID')
ncaa_aggdiff.to_csv('./GeneratedDatasets/NCAA-AggDiff.csv', index_label='ID')

In [283]:
ncaa_agg.shape

(981, 145)

In [284]:
ncaa_diff.shape

(981, 74)

In [285]:
ncaa_aggdiff.shape

(981, 216)

In [286]:
seeds = pd.read_csv('./Stage2UpdatedDataFiles/NCAATourneySeeds.csv')

In [392]:
tourney_teamlist = list(seeds[seeds['Season'] == 2018].sort_values('TeamID')['TeamID'].values)

In [393]:
tourney_teamlist

[1104,
 1112,
 1113,
 1116,
 1120,
 1137,
 1138,
 1139,
 1153,
 1155,
 1158,
 1166,
 1168,
 1172,
 1181,
 1196,
 1199,
 1209,
 1211,
 1222,
 1233,
 1242,
 1243,
 1246,
 1252,
 1254,
 1260,
 1267,
 1274,
 1276,
 1277,
 1281,
 1285,
 1293,
 1300,
 1301,
 1305,
 1308,
 1314,
 1326,
 1328,
 1335,
 1344,
 1345,
 1347,
 1348,
 1355,
 1361,
 1371,
 1372,
 1382,
 1393,
 1395,
 1397,
 1400,
 1401,
 1403,
 1411,
 1417,
 1420,
 1422,
 1437,
 1438,
 1439,
 1452,
 1455,
 1460,
 1462]

In [346]:
def gen_target_agg(teamlist):
    team1 = teamlist.pop(0)
    team2 = teamlist.pop(0)
    df_agg = gen_aggregate(team1, team2, 2018)
    df_target_agg = df_agg
    for t in teamlist:
        df_agg = gen_aggregate(team1, t, 2018)
        df_target_agg = pd.concat([df_target_agg, df_agg])
    for t in teamlist:
        df_agg = gen_aggregate(team2, t, 2018)
        df_target_agg = pd.concat([df_target_agg, df_agg])
    while len(teamlist) > 1:
        team1 = teamlist.pop(0)
        for t in teamlist:
            df_agg = gen_aggregate(team1, t, 2018)
            df_target_agg = pd.concat([df_target_agg, df_agg])
    return df_target_agg

In [361]:
gen_target_agg(tourney_teamlist).to_csv('./GeneratedDatasets/2018Target-Aggregate.csv', index_label='ID')

In [340]:
def gen_target_diff(teamlist):
    team1 = teamlist.pop(0)
    team2 = teamlist.pop(0)
    df_diff = gen_differential(team1, team2, 2018)
    df_target_diff = df_diff
    for t in teamlist:
        df_diff = gen_differential(team1, t, 2018)
        df_target_diff = pd.concat([df_target_diff, df_diff])
    for t in teamlist:
        df_diff = gen_differential(team2, t, 2018)
        df_target_diff = pd.concat([df_target_diff, df_diff])
    while len(teamlist) > 1:
        team1 = teamlist.pop(0)
        for t in teamlist:
            df_diff = gen_differential(team1, t, 2018)
            df_target_diff = pd.concat([df_target_diff, df_diff])
    return df_target_diff

In [364]:
gen_target_diff(tourney_teamlist).to_csv('./GeneratedDatasets/2018Target-Differential.csv', index_label='ID')

In [394]:
def gen_target_aggdiff(teamlist):
    team1 = teamlist.pop(0)
    team2 = teamlist.pop(0)
    df_agg = gen_aggregate(team1, team2, 2018)
    df_diff = gen_differential(team1, team2, 2018)
    df_target_aggdiff = df_agg.join(df_diff)
    for t in teamlist:
        df_agg = gen_aggregate(team1, t, 2018)
        df_diff = gen_differential(team1, t, 2018)
        df_target_aggdiff = pd.concat([df_target_aggdiff, df_agg.join(df_diff)])
    for t in teamlist:
        df_agg = gen_aggregate(team2, t, 2018)
        df_diff = gen_differential(team2, t, 2018)
        df_target_aggdiff = pd.concat([df_target_aggdiff, df_agg.join(df_diff)])
    while len(teamlist) > 1:
        team1 = teamlist.pop(0)
        for t in teamlist:
            df_agg = gen_aggregate(team1, t, 2018)
            df_diff = gen_differential(team1, t, 2018)
            df_target_aggdiff = pd.concat([df_target_aggdiff, df_agg.join(df_diff)])
    return df_target_aggdiff

In [395]:
gen_target_aggdiff(tourney_teamlist).to_csv('./GeneratedDatasets/2018Target-AggDiff.csv', index_label='ID')